# 🚆Train_station
지하철 정류장 데이터 전처리

### TODO
- [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
- [X] 결측값 확인
  - [X] 위경도 결측값 분석
  - [X] 결측값 채워진 데이터셋 구하기
  - [X] 노선명 통일
  - [X] 역사명 통일
- [X] 두 데이터셋 하나로 결합
  - [X] 위경도 이상값 확인 
  - [X] 위경도 결측값 재확인
  - [ ] ~~역번호 이상값 확인~~
    - 추후 지하철 시간표와 같은 값으로 선택 필요
- [X] 지도에 시각화
  - [X] 이상값 확인
- [ ] ~~수도권 바깥 데이터 삭제~~
  - 추후 버스역 데이터까지 불러온 뒤 적절히 선택 필요
- [ ] 역번호, 노선번호, 역이름, 노선명 맞추기
  - [X] 서울시 지하철 실시간 도착정보에 쓰이는 역 종류 확인
  - [X] 전체 지하철 시간표 정보에 쓰이는 역 종류 확인
  - [ ] 역번호 통일
  - [ ] 노선번호 통일 
  - [ ] 역이름 통일
  - [ ] 노선명 통일 
    - [X] 노선명 일차 통일
    - [ ] 수인분당선, 경의선 해결
- [ ] 환승역, 종점, 분기역 정보 가공

### 0. import

In [159]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium

### 1. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [160]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436
...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


### 2. 결측값 확인

##### 2-a. 위경도 결측값 분석

In [161]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [162]:
# 결측값이 컬럼별로 몇 개인지 확인
station.isnull().sum()

역번호         0
역사명         0
노선번호        0
노선명         0
환승역구분       0
환승노선번호    476
환승노선명     476
역위도       352
역경도       352
dtype: int64

In [163]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

##### 2-b. 결측값 채워진 데이터셋 구하기

In [164]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 결합 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
station_seoul = station_seoul[['역번호', '역사명', '노선명', '위도', '경도']]

# 필요없는 노선 제거
station_seoul = station_seoul[(station_seoul['노선명'] != '장항선')]

station_seoul

,역번호,역사명,노선명,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849
761,153,종로3가,1호선,37.570406,126.991847
762,152,종각,1호선,37.570161,126.982923
763,151,시청,1호선,37.565715,126.977088


In [165]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [166]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

역번호    0
역사명    0
노선명    0
위도     0
경도     0
dtype: int64

##### 2-c. 노선명 통일

In [167]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'인천국제공항선', '에버라인', '수도권 경량도시철도 신림선', '도시철도 7호선', '의정부', '인천지하철 1호선', '인천지하철 2호선', '안산과천선', '수도권  도시철도 9호선'}
{'과천선', '중앙선', '의정부선', '인천2호선', '에버라인선', '신분당선(연장2)', '9호선', '신분당선(연장)', '안산선', '인천1호선', '7호선(인천)', '9호선(연장)', '공항철도1호선', '신림선'}


In [168]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도1호선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 안산과천선의 경우, 역사명 통일한 이후 안산선 / 과천선으로 나눌 예정

{'안산과천선'}
{'과천선', '중앙선', '신분당선(연장2)', '신분당선(연장)', '안산선', '7호선(인천)', '9호선(연장)'}


##### 2-d. 역사명 통일

In [169]:
# 둘 중 한 데이터셋에만 있는 역사명 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['가능역', '가산디지털단지역', '가좌역', '가천대역', '가평역', '간석역', '갈매역', '강남구청역', '강매역', '강촌역', '개봉역', '개포동역', '걸포북변역', '검단오류', '경기광주역', '경마공원역', '고색역', '고잔역', '고촌역', '곡산역', '곤지암역', '공덕역', '과천역', '관악산', '관악역', '광명역', '광운대역', '교대(법원·검찰청)', '구래역', '구로역', '구룡역', '구리역', '구성역', '구일역', '국수역', '군포역', '굴봉산역', '금곡역', '금릉역', '금정역', '금천구청역', '금촌역', '기흥(백남준아트센터)', '기흥역', '김유정역', '김포공항역', '낙성대(강감찬)', '남동인더스파크역', '남영역', '남춘천역', '남한산성입구(성남법원·검찰청)', '노량진역', '녹양역', '녹천역', '능곡역', '달미역', '달월역', '당정역', '대곡역', '대공원역', '대모산입구역', '대방역', '대성리역', '대야미역', '대화역', '덕계역', '덕소역', '덕정역', '도곡역', '도농역', '도봉산역', '도봉역', '도심역', '도원역', '도화역', '독산역', '동대문역사문화공원(DDP)', '동두천역', '동두천중앙역', '동백(용인세브란스)', '동암역', '동인천역', '동작', '두정역', '디지털미디어시티역', '마두역', '마산역', '마석역', '망우역', '망월사역', '망포역', '매교역', '매탄권선역', '명학역', '모란역', '문산역', '미금역', '반월역', '방학역', '배방역', '백마역', '백석역', '백양리역', '백운역', '범계역', '별내별가람역', '별내역', '병점역', '보산역', '보정역', '복정역', '봉명역', '부개역', '부발역', '부천역', '부평역', '사릉역', '사리역', '사우(김포시청)역', '산본역', '삼동역', '삼송역', '상갈역', '상록수역', '상봉역', '상천역', '서강대

In [170]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['검단오류', '관악산', '교대(법원·검찰청)', '기흥(백남준아트센터)', '낙성대(강감찬)', '남한산성입구(성남법원·검찰청)', '동대문역사문화공원(DDP)', '동백(용인세브란스)', '동작', '배방', '봉명', '상봉', '서울역(경의선)', '성신여대입구', '수원(분당)', '시민공원', '시우', '시청·용인대', '신창(순천향대)', '신촌(지하)', '쌍용(나사렛대)', '아산', '아시아드경기장', '온수', '온양온천', '왕십리', '용마산(용마폭포공원)', '운동장·송담대(중앙시장)', '운연', '이촌', '전대·에버랜드', '종로3가(탑골공원)', '청량리', '탕정', '하남시청(덕풍∙신장)', '흑석']
['검단오류(검단산업단지)', '관악산(서울대)', '교대(법원.검찰청)', '낙성대', '남한산성입구(성남법원.검찰청)', '동대문역사문화공원', '동백', '둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '시민공원(문화창작지대)', '시청.용인대', '아시아드경기장(공촌사거리)', '언주', '용마산', '운동장.송담대', '운연(서창)', '원곡', '이수', '전대.에버랜드', '중앙보훈병원', '하남시청(덕풍-신장)', '한성백제', '흑석(중앙대입구)']


In [171]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)"
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)"
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['기흥(백남준아트센터)', '배방', '봉명', '상봉', '성신여대입구', '시우', '신창(순천향대)', '쌍용(나사렛대)', '아산', '온수', '온양온천', '왕십리', '이촌', '종로3가(탑골공원)', '청량리', '탕정']
['둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '언주', '원곡', '이수', '중앙보훈병원', '한성백제']


In [172]:
# 안산과천선 -> 안산선 / 과천선
station_name_ansan = set(station_seoul['역사명'][station_seoul['노선명'] == '안산선']) 
station_name_gwacheon = set(
    station_seoul['역사명'][station_seoul['노선명'] == '과천선'])

station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_ansan)), "노선명"] = "안산선"
station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_gwacheon)), "노선명"] = "과천선"

# 잘 바뀌었는지 확인
station["노선명"].value_counts().sort_index()

1호선        10
2호선        50
3호선        34
4호선        26
5호선        56
6호선        39
7호선        53
8호선        18
9호선        25
경강선        11
경부선        45
경원선        24
경의중앙선      57
경인선        20
경춘선        24
공항철도1호선    14
과천선         8
김포골드라인     10
분당선        37
서해선        12
수인선        26
신림선        11
신분당선       16
안산선        13
에버라인선      15
우이신설선      13
의정부선       15
인천1호선      30
인천2호선      27
일산선        11
진접선         3
Name: 노선명, dtype: int64

### 3. 두 데이터셋 하나로 결합

##### 3-a. 위경도값 하나로 합치기

In [173]:
# 일단 하나로 합쳐보기
station_result = pd.merge(station, station_seoul, on=['역사명', '노선명'], how='outer')
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...
822,NaN,옥수,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1011.0,37.540446,127.018672
823,NaN,한남,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1010.0,37.529430,127.009169
824,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1009.0,37.519594,126.988537
825,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,NaN,NaN,1008.0,37.522427,126.973406


In [174]:
# x, y에 대해, 둘 중 한 곳에 없는 위경도값이 각각 몇 개인지 count
lat_in_only_x = station_result[station_result["위도_y"].isnull(
) & station_result["위도_x"].notnull()]
lng_in_only_x = station_result[station_result["경도_y"].isnull(
) & station_result["경도_x"].notnull()]

lat_in_only_x_cnt = len(lat_in_only_x)
lng_in_only_x_cnt = len(lng_in_only_x)

print("x에만 있는 위경도값 개수 :", lat_in_only_x_cnt, lng_in_only_x_cnt)

lat_in_only_y = station_result[station_result["위도_x"].isnull(
) & station_result["위도_y"].notnull()]
lng_in_only_y = station_result[station_result["경도_x"].isnull(
) & station_result["경도_y"].notnull()]

lat_in_only_y_cnt = len(lat_in_only_y)
lng_in_only_y_cnt = len(lng_in_only_y)

print("y에만 있는 위경도값 개수 :", lat_in_only_y_cnt, lng_in_only_y_cnt)

# 한쪽 위경도가 NaN이면, 반대쪽 위경도 넣어주기
station_result["위도_x"] = station_result["위도_x"].fillna(station_result["위도_y"])
station_result["위도_y"] = station_result["위도_y"].fillna(station_result["위도_x"])
station_result["경도_x"] = station_result["경도_x"].fillna(station_result["경도_y"])
station_result["경도_y"] = station_result["경도_y"].fillna(station_result["경도_x"])

# 위경도 데이터의 오차 계산
station_result["위도차"] = (station_result["위도_x"] - station_result["위도_y"]).abs()
station_result["경도차"] = (station_result["경도_x"] - station_result["경도_y"]).abs()

x에만 있는 위경도값 개수 : 66 66
y에만 있는 위경도값 개수 : 419 419


In [175]:
# 위도 오차값 확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff

0.6563686820821601


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
113,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307.0,37.496837,127.028104,89.531042,89.530719
114,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308.0,37.483809,127.034653,89.550290,89.550199
115,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309.0,37.470023,127.038420,89.568397,89.567456
116,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310.0,37.447211,127.055664,89.607129,89.606949
117,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311.0,37.394761,127.112217,89.716450,89.716502
118,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312.0,37.367098,127.108403,89.741288,89.740193
710,1886,송도,I28K1,수인선,일반역,NaN,NaN,34.429705,126.654401,1886.0,37.428514,126.657772,2.998809,0.003371


In [176]:
# 경도 오차값 확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff

0.656387570673301


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
54,3225,남동구청,S2802,인천2호선,일반역,-,-,37.448179,123.736581,3225.0,37.448161,126.736939,0.000018,3.000358
113,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307.0,37.496837,127.028104,89.531042,89.530719
114,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308.0,37.483809,127.034653,89.550290,89.550199
115,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309.0,37.470023,127.038420,89.568397,89.567456
116,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310.0,37.447211,127.055664,89.607129,89.606949
117,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311.0,37.394761,127.112217,89.716450,89.716502
118,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312.0,37.367098,127.108403,89.741288,89.740193


In [177]:
# 113 ~ 118, 위도_x와 경도_x가 뒤바뀌어 들어감
# 송도, (위도_x, 경도_x) 전남 해남에 위치 
# 정자, (위도_x, 경도_x) 황해에 위치
# -> 위경도 데이터의 경우, x를 y로 덮어쓰면 될 것 같음
# 이를 증명하기 위해, 위의 데이터들의 위도차/경도차가 해결된 뒤, 
# 위경도차가 높은 행들을 각각 확인하여, 거리상의 오차가 어느 정도일지 확인해볼 것

station_result.loc[[113, 114, 115, 116, 117, 118, 710, 54], ['위도차', '경도차']] = 0.0

In [178]:
# 위도 오차값 평균 재확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff.sort_values('위도차').tail()

0.00016307176662614506


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
740,1952,화정,I4106,일산선,일반역,NaN,NaN,37.637837,126.832503,1952.0,37.634592,126.832650,0.003245,0.000147
758,4815,원시,I41WS,서해선,일반역,NaN,NaN,37.305947,126.788297,4815.0,37.302371,126.786691,0.003576,0.001606
44,3215,인천가좌,S2802,인천2호선,일반역,-,-,37.493762,126.675465,3215.0,37.489700,126.675208,0.004062,0.000257
83,A01,서울역,I28A1,공항철도1호선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,4201.0,37.553247,126.969769,0.004129,0.000695
563,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268.0,37.602888,126.868387,0.034949,0.035884


In [179]:
# 경도 오차값 평균 재확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff.sort_values('경도차').tail()

0.00017913939902904088


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
36,3207,검암,S2802,인천2호선,환승역,S2802+I28A1,공항철도,37.568611,126.671221,3207.0,37.568660,126.675687,0.000049,0.004466
468,1502,이매,I41K5,경강선,환승역,I28K1,분당선,37.394736,127.123048,1502.0,37.394655,127.127819,0.000081,0.004771
692,1876,야목,I28K1,수인선,일반역,NaN,NaN,37.261011,126.884290,1876.0,37.264179,126.879483,0.003168,0.004807
96,A11,인천공항2터미널,I28A1,공항철도1호선,일반역,NaN,NaN,37.458269,126.427536,4215.0,37.460699,126.441442,0.002430,0.013906
563,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268.0,37.602888,126.868387,0.034949,0.035884


##### 3-b. 위경도 결측값 재확인

In [180]:
# 화전, (위도_x, 경도_x) 덕양구청에 위치, 도보 1시간 30분의 오차
# 그 다음으로 오차가 큰 인천공항2터미널과 서울역의 경우 역이 워낙 커서 생기는 일, 문제 없음
# 즉, x와 y 두 곳에 있는 값들에 대해서는 이상값 검토가 끝남 
# 둘 다에 있는 값의 경우 그냥 y쪽 값으로 덮어쓰기
# 둘 중 한 곳에만 있는 위경도값들은 위에서 따로 저장해 둠, 
# 이후 지도를 보며 재검토할 예정  

# y에 없는 값들 x에서 가져와 채우기
station_result['위도_y'] = station_result['위도_y'].fillna(station_result['위도_x'])
station_result['경도_y'] = station_result['경도_y'].fillna(station_result['경도_x'])

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

null_lat_in_y 

y에 있는 위경도 결측값 개수 : 3 3


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
107,S120,성신여대입구,L11UI,우이신설선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,0534,종로3가(탑골공원),S1105,5호선,도시철도 환승역,I1103+I1101,수도권 광역철도 3호선+수도권 광역철도 1호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,0736,총신대입구(이수),S1107,7호선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
# 성신여대입구, 종로3가, 총신대입구에 대한 위경도 정보 y쪽에 수동으로 넣어주기 

station_result.loc[107, "위도_y"] = 37.592359
station_result.loc[107, "경도_y"] = 127.016551 

station_result.loc[294, "위도_y"] = 37.572571
station_result.loc[294, "경도_y"] = 126.990293

station_result.loc[394, "위도_y"] = 37.485263
station_result.loc[394, "경도_y"] = 126.981609

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

y에 있는 위경도 결측값 개수 : 0 0


In [182]:
# 위도_x, 경도_x 삭제 및 위도_y와 경도_y의 이름 변경
station_result.drop(columns=["위도_x", "경도_x", "위도차", "경도차"], inplace=True)
station_result.rename(columns={'위도_y': '위도', '경도_y': '경도'}, inplace=True)

station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...
822,NaN,옥수,NaN,경원선,NaN,NaN,NaN,1011.0,37.540446,127.018672
823,NaN,한남,NaN,경원선,NaN,NaN,NaN,1010.0,37.529430,127.009169
824,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,1009.0,37.519594,126.988537
825,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,1008.0,37.522427,126.973406


### 4. 지도에 시각화

##### 4-a. 이상값 확인

In [183]:
# 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=0, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
        " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']
    
    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name, 
        tooltip=popup_name
    ).add_to(fmap)

fmap

In [184]:
# 시각화 결과, 광교 등 몇몇 역의 위경도 좌표가 반대로 들어가 있는 것처럼 보임
# 위경도가 한국을 벗어난 역들 찾아보기
station_outof_korea = station_result[(station_result['위도'] < 33.10000000) |
                                     (station_result['위도'] > 38.45000000) | (station_result['경도'] < 124.19583333) |
                                     (station_result['경도'] > 131.87222222)]

station_outof_korea

# 확인 결과, 범위를 벗어나는 역 전부 위경도 데이터가 반대로 들어가 있음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도
110,D004,신사,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,NaN,127.019761,37.516125
111,D005,논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,NaN,127.021366,37.511115
112,D006,신논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,NaN,127.024531,37.504555
119,D013,미금,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,NaN,127.108903,37.351183
120,D014,동천,I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.102794,37.338684
121,D015,수지구청,I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.095581,37.323009
122,D016,성복,I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.080758,37.314043
123,D017,상현,I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.069580,37.298685
124,D018,광교중앙(아주대),I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.052625,37.287738
125,D019,광교(경기대),I11D1,신분당선,도시철도 일반역,NaN,NaN,NaN,127.044856,37.301735


In [185]:
# 위경도 뒤집어서 넣어주기
station_result.loc[station_outof_korea.index, '위도'] = station_outof_korea['경도']
station_result.loc[station_outof_korea.index, '경도'] = station_outof_korea['위도']

In [186]:
# 위경도가 한국을 벗어난 역들 다시 찾아보기
station_outof_korea = station_result[(station_result['위도'] < 33.10000000) |
                                     (station_result['위도'] > 38.45000000) | (station_result['경도'] < 124.19583333) |
                                     (station_result['경도'] > 131.87222222)]

station_outof_korea

# 확인 결과, 모든 역이 제대로 한국 내에 존재함

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도


In [187]:
# 바뀐 데이터로 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=7, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']
                                                            ) + " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

In [188]:
# 확인 결과, 경의중앙선의 양원역 위치가 경북 봉화군에 위치한 양원역 위치로 잘못 들어가져 있음 확인
# 수동으로 데이터 수정 필요
station_result.loc[579, "위도"] = 37.606551
station_result.loc[579, "경도"] = 127.107661

station_result[station_result["역사명"] == "양원"]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도
579,1204,양원,I4108,경의중앙선,일반역,NaN,NaN,NaN,37.606551,127.107661
814,NaN,양원,NaN,중앙선,NaN,NaN,NaN,1204.0,37.606596,127.107906


In [189]:
# 바뀐 데이터로 다시 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=8, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']
                                                            ) + " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

### 5. 역번호, 노선번호, 역이름, 노선명 맞추기

##### 5-a. 서울시 지하철 실시간 도착정보에 쓰이는 역 및 노선 종류 확인

In [190]:
# 데이터 읽어오기
station_num_name_seoul = pd.read_csv('train_station_num_name_seoul.csv')

# 컬럼명 변경
station_num_name_seoul.columns=["역번호", "역사명", "노선명", "외부코드"]

station_num_name_seoul

,역번호,역사명,노선명,외부코드
0,244,용답,02호선,211-1
1,245,신답,02호선,211-2
2,250,용두,02호선,211-3
3,336,학여울,03호선,346
4,428,삼각지,04호선,428
...,...,...,...,...
762,159,동묘앞,01호선,127
763,200,까치산,02호선,234-4
764,201,시청,02호선,201
765,202,을지로입구,02호선,202


In [191]:
# 결측값 존재 확인
station_num_name_seoul.info()

# 확인 결과, 외부코드를 제외한 모든 값이 잘 들어있음 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역번호     767 non-null    object
 1   역사명     767 non-null    object
 2   노선명     767 non-null    object
 3   외부코드    756 non-null    object
dtypes: object(4)
memory usage: 24.1+ KB


In [192]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

# 확인 결과, 공항철도, 경춘선 이름을 맞출 필요 있음
# 수인분당선, 경의선의 경우 다른, 다른 모든 API 확인한 뒤 통합할지 쪼갤지 고민 필요

{'과천선', '중앙선', '신분당선(연장2)', '수인선', '우이신설선', '경원선', '신림선', '경춘선', '7호선', '서해선', '8호선', '3호선', '에버라인선', '6호선', '7호선(인천)', '공항철도1호선', '2호선', '의정부선', '인천2호선', '9호선', '신분당선(연장)', '1호선', '안산선', '진접선', '일산선', '5호선', '4호선', '김포골드라인', '경의중앙선', '경부선', '인천1호선', '분당선', '신분당선', '9호선(연장)', '경인선', '경강선'}
{'우이신설경전철', '09호선', '01호선', '신림선', '경춘선', '서해선', '06호선', '의정부경전철', '07호선', '공항철도', '02호선', '05호선', '수인분당선', '인천2호선', '04호선', '경강선', '김포도시철도', '경의선', '인천선', '신분당선', '용인경전철', '08호선', '03호선'}
{'과천선', '중앙선', '신분당선(연장2)', '수인선', '우이신설선', '경원선', '7호선', '8호선', '3호선', '에버라인선', '6호선', '7호선(인천)', '공항철도1호선', '2호선', '의정부선', '9호선', '신분당선(연장)', '1호선', '안산선', '진접선', '일산선', '5호선', '4호선', '김포골드라인', '경의중앙선', '경부선', '인천1호선', '분당선', '9호선(연장)', '경인선'}
{'경의선', '수인분당선', '인천선', '06호선', '의정부경전철', '07호선', '04호선', '08호선', '우이신설경전철', '03호선', '용인경전철', '09호선', '01호선', '공항철도', '02호선', '김포도시철도', '05호선'}


In [193]:
# 수인분당선, 경의중앙선을 제외한 부분만 일단 통일
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].replace({
    "우이신설경전철": "우이신설선",
    "의정부경전철": "의정부선",
    "용인경전철": "에버라인선",
    "김포도시철도": "김포골드라인",
    "인천선": "인천1호선"
})
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].str.replace(
    pat=r'^0', repl=r'', regex=True)
station_result["노선명"] = station_result["노선명"].replace({
    "공항철도1호선": "공항철도"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

{'과천선', '중앙선', '신분당선(연장2)', '수인선', '우이신설선', '경원선', '신림선', '경춘선', '7호선', '서해선', '8호선', '3호선', '에버라인선', '6호선', '7호선(인천)', '공항철도', '2호선', '의정부선', '인천2호선', '9호선', '신분당선(연장)', '1호선', '안산선', '진접선', '일산선', '5호선', '4호선', '김포골드라인', '경의중앙선', '경부선', '인천1호선', '분당선', '신분당선', '9호선(연장)', '경인선', '경강선'}
{'우이신설선', '신림선', '경춘선', '7호선', '서해선', '8호선', '3호선', '에버라인선', '6호선', '공항철도', '수인분당선', '2호선', '의정부선', '인천2호선', '9호선', '1호선', '5호선', '4호선', '김포골드라인', '경의선', '인천1호선', '신분당선', '경강선'}
{'과천선', '중앙선', '경부선', '신분당선(연장2)', '신분당선(연장)', '안산선', '진접선', '7호선(인천)', '일산선', '분당선', '9호선(연장)', '수인선', '경인선', '경원선', '경의중앙선'}
{'경의선', '수인분당선'}


##### 5-b. 전체 지하철 시간표 정보에 쓰이는 역 및 노선 종류 확인

In [194]:
# 데이터 읽어오기
station_num_name = pd.read_csv('train_station_num_name.csv')

# 컬럼명 변경
station_num_name = station_num_name.rename(
    columns={"LN_CD": "노선번호", "LN_NM": "노선명", "STIN_CD": "역번호", "STIN_NM": "역사명"})

# 필요 없는 노선 제거
station_num_name = station_num_name[(station_num_name['노선명'] != '동해선')]
station_num_name = station_num_name[(station_num_name['노선명'] != '부산김해경전철')]
station_num_name = station_num_name[(station_num_name['노선명'] != '자기부상')]

station_num_name = station_num_name.reset_index(drop=True)
station_num_name  

,RAIL_OPR_ISTT_CD,RAIL_OPR_ISTT_NM,노선번호,노선명,역번호,역사명
0,AR,공항철도주식회사,A1,공항철도,A01,서울역
1,AR,공항철도주식회사,A1,공항철도,A02,공덕
2,AR,공항철도주식회사,A1,공항철도,A03,홍대입구
3,AR,공항철도주식회사,A1,공항철도,A04,디지털미디어시티
4,AR,공항철도주식회사,A1,공항철도,A042,마곡나루
...,...,...,...,...,...,...
1011,DJ,대전교통공사,1,1호선,0018,현충원(한밭대)
1012,DJ,대전교통공사,1,1호선,0019,월드컵경기장(노은도매시장)
1013,DJ,대전교통공사,1,1호선,0020,노은
1014,DJ,대전교통공사,1,1호선,0021,지족(침신대)


In [195]:
# 결측값 존재 확인
station_num_name.info()

# 확인 결과, 모든 값이 제대로 잘 들어있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   RAIL_OPR_ISTT_CD  1016 non-null   object
 1   RAIL_OPR_ISTT_NM  1016 non-null   object
 2   노선번호              1016 non-null   object
 3   노선명               1016 non-null   object
 4   역번호               1016 non-null   object
 5   역사명               1016 non-null   object
dtypes: object(6)
memory usage: 47.8+ KB


In [196]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name = set(station_num_name["노선명"])
sub_route_name_result = route_name_result - route_name
sub_route_name = route_name - route_name_result

print(sub_route_name_result)
print(sub_route_name)

{'과천선', '중앙선', '신분당선(연장2)', '수인선', '우이신설선', '경원선', '경춘선', '에버라인선', '7호선(인천)', '의정부선', '신분당선(연장)', '안산선', '진접선', '일산선', '경의중앙선', '경부선', '분당선', '9호선(연장)', '경인선'}
{'의정부경전철', '용인에버라인', '수인분당', '경춘', '경의중앙', '우이신설'}


In [197]:
# 수인분당선, 경의중앙선을 제외한 부분만 일단 통일
station_num_name["노선명"] = station_num_name["노선명"].replace({
    "의정부경전철": '의정부선',
    "용인에버라인": "에버라인선",
    "수인분당": "수인분당선",
    "경춘": "경춘선",
    "경의중앙": "경의중앙선", 
    "우이신설": "우이신설선"
})

# 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name = set(station_num_name["노선명"])
sub_route_name_result = route_name_result - route_name
sub_route_name = route_name - route_name_result

print(sub_route_name_result)
print(sub_route_name)

{'과천선', '중앙선', '경부선', '신분당선(연장2)', '신분당선(연장)', '안산선', '진접선', '7호선(인천)', '일산선', '분당선', '9호선(연장)', '수인선', '경인선', '경원선'}
{'수인분당선'}
